In [1]:
import pandas as pd
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import linear_kernel
import pickle
from eunjeon import Mecab
import torch

mecab=Mecab() #형태소 분해 객체 생성

In [2]:
file_path = r'./data/clean_laws_jo_total.json'
with open(file_path, 'r', encoding="UTF-8") as jsonfile:
    json_data = json.load(jsonfile)  

In [3]:
laws_data_list = json_data["laws"]
len(laws_data_list)

145279

In [4]:
def tokenizer(raw, pos = ['NNG', 'VV', 'VA+ETM','XSV+EC']):
    mecab = Mecab()
    # 길이가 1 이하인 토근은 제외, 위에서 지정한 (Okt 사전에 따른) 토큰들만 특징으로 삼기  
    return [word for word, tag in mecab.pos(raw) if len(word) > 1 and tag in pos]

In [5]:
# TF-IDF 불러오기
with open("tfidf_fit.pickle", 'rb') as f:
    load_tfidf_fit = pickle.load(f) 

In [6]:
# TF-IDF transform불러오기
with open("tfidf_vectors.pickle", 'rb') as f:
    load_tfidf_vectors = pickle.load(f) 

In [7]:
features = load_tfidf_fit.get_feature_names() 

C:\Users\w0w12\.conda\envs\py3.8\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
input_text_data =  '등록금 건축적립금 관리'

In [9]:
srch = [ t for t in tokenizer(input_text_data) if t in features]
srch

['등록금', '건축', '적립금', '관리']

In [10]:
srch_vector = load_tfidf_fit.transform([input_text_data])

In [11]:
cosine_similar =  linear_kernel(srch_vector, load_tfidf_vectors).flatten()

In [12]:
sim_rank_idx = cosine_similar.argsort()[::-1]

In [13]:
#tfidf result
tf_idf_result_index = [] #실 데이터 인덱스
tf_idf_sentences = []# 결과 조문
for i in sim_rank_idx:
    if cosine_similar[i] > 0.13:
        print('{} /score : {}'.format(laws_data_list[i],cosine_similar[i]))
        tf_idf_result_index.append(i)
        tf_idf_sentences.append(laws_data_list[i])

제11조등록금 및 등록금심의위원회  학교의 설립자경영자는 수업료와 그 밖의 납부금이하 등록금이라 한다을 현금 또는 여신전문금융업법 제2조에 따른 신용카드 직불카드 선불카드에 의한 결제로 납부 받을 수 있다 이 경우 학생은 학칙으로 정하는 바에 따라 해당 학기에 납부하여야 할 등록금을 2회 이상으로 분할하여 납부할 수 있다    제1항에도 불구하고 학교제30조에 따른 대학원대학은 제외한다의 설립자경영자는 해당 학교에 입학 또는 편입학하는 사람제29조에 따라 대학원에 두는 학위과정 연구과정 및 제29조의3에 따라 통합된 학위과정에 입학 또는 편입학하는 사람은 제외한다으로부터 입학금을 받을 수 없다    각 학교는 등록금을 책정하기 위하여 교직원사립대학의 경우에는 학교법인이 추천하는 재단인사를 포함한다 학생 관련 전문가 등으로 구성되는 등록금심의위원회를 설치운영하여야 한다 이 경우 학생 위원은 전체 위원 정수의 10분의 3 이상 구성단위별 위원은 10분의 5 미만이 되도록 하고 관련 전문가 위원을 선임할 때에는 학칙으로 정하는 바에 따라 학교를 대표하는 측과 학생을 대표하는 측이 협의하여야 한다    학교는 재난 및 안전관리 기본법 제3조제1호에 따른 재난으로 인하여 학교시설의 이용 및 실험실습이 제한되거나 수업시수가 감소하는 등 학사운영이 정상적으로 이루어지지 아니한 경우 등록금을 면제감액할 수 있다    학교는 특별한 사정이 없으면 등록금심의위원회의 심의결과를 최대한 반영하여야 한다    제3항의 등록금심의위원회는 교육관련기관의 정보공개에 관한 특례법 제6조제1항제8호의2의 등록금 및 학생 1인당 교육비 산정근거 도시근로자 평균가계소득 제7조제3항의 고등교육 지원계획 등록금 의존율대학교육비에서 등록금이 차지하는 비율을 말한다 등을 감안하여 해당 연도의 등록금을 적정하게 산정하여야 한다    제4항에 따른 등록금의 면제감액 규모는 등록금심의위원회에서 논의하여야 한다    등록금심의위원회는 등록금 산정을 위하여 필요한 경우 대통령령으로 정하는 바에 따라 학교의 장에

In [14]:
len(tf_idf_sentences)

351

In [15]:
sbert_result_index = []
from sentence_transformers import SentenceTransformer, util
import numpy as np
# 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
# "jhgan/ko-sroberta-multitask"
embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

# TF-IDF 결과 조문을 Corpus로
corpus = tf_idf_sentences
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
q_list = []
q_list.append(input_text_data)
queries = q_list

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 20
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

     #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))
        sbert_result_index.append(tf_idf_result_index[idx])

C:\Users\w0w12\.conda\envs\py3.8\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0






Query: 등록금 건축적립금의 관리

Top 5 most similar sentences in corpus:
제11조등록금 및 등록금심의위원회  학교의 설립자경영자는 수업료와 그 밖의 납부금이하 등록금이라 한다을 현금 또는 여신전문금융업법 제2조에 따른 신용카드 직불카드 선불카드에 의한 결제로 납부 받을 수 있다 이 경우 학생은 학칙으로 정하는 바에 따라 해당 학기에 납부하여야 할 등록금을 2회 이상으로 분할하여 납부할 수 있다    제1항에도 불구하고 학교제30조에 따른 대학원대학은 제외한다의 설립자경영자는 해당 학교에 입학 또는 편입학하는 사람제29조에 따라 대학원에 두는 학위과정 연구과정 및 제29조의3에 따라 통합된 학위과정에 입학 또는 편입학하는 사람은 제외한다으로부터 입학금을 받을 수 없다    각 학교는 등록금을 책정하기 위하여 교직원사립대학의 경우에는 학교법인이 추천하는 재단인사를 포함한다 학생 관련 전문가 등으로 구성되는 등록금심의위원회를 설치운영하여야 한다 이 경우 학생 위원은 전체 위원 정수의 10분의 3 이상 구성단위별 위원은 10분의 5 미만이 되도록 하고 관련 전문가 위원을 선임할 때에는 학칙으로 정하는 바에 따라 학교를 대표하는 측과 학생을 대표하는 측이 협의하여야 한다    학교는 재난 및 안전관리 기본법 제3조제1호에 따른 재난으로 인하여 학교시설의 이용 및 실험실습이 제한되거나 수업시수가 감소하는 등 학사운영이 정상적으로 이루어지지 아니한 경우 등록금을 면제감액할 수 있다    학교는 특별한 사정이 없으면 등록금심의위원회의 심의결과를 최대한 반영하여야 한다    제3항의 등록금심의위원회는 교육관련기관의 정보공개에 관한 특례법 제6조제1항제8호의2의 등록금 및 학생 1인당 교육비 산정근거 도시근로자 평균가계소득 제7조제3항의 고등교육 지원계획 등록금 의존율대학교육비에서 등록금이 차지하는 비율을 말한다 등을 감안하여 해당 연도의 등록금을 적정하게 산정하여야 한다    제4항에 따른 등록금의 면제감액 규모는 등록금심의위원회에

In [16]:
cos_scores

tensor([0.6641, 0.5340, 0.4684, 0.5128, 0.5565, 0.4638, 0.4603, 0.4006, 0.5165,
        0.6397, 0.3520, 0.4257, 0.4843, 0.4877, 0.4363, 0.3887, 0.5829, 0.4095,
        0.4823, 0.5006, 0.3720, 0.5995, 0.4242, 0.5524, 0.3361, 0.3022, 0.3751,
        0.3647, 0.5161, 0.4901, 0.3289, 0.3860, 0.4048, 0.4845, 0.3295, 0.4823,
        0.3297, 0.4205, 0.3771, 0.2821, 0.3925, 0.4570, 0.4193, 0.3997, 0.2916,
        0.4982, 0.4586, 0.4185, 0.4039, 0.3969, 0.3808, 0.3620, 0.3370, 0.4295,
        0.4366, 0.3978, 0.2911, 0.3816, 0.4836, 0.3848, 0.3983, 0.4898, 0.4393,
        0.3936, 0.3926, 0.4123, 0.4927, 0.3938, 0.2960, 0.3844, 0.3724, 0.3498,
        0.3692, 0.3825, 0.2990, 0.6041, 0.2961, 0.4095, 0.3660, 0.3525, 0.4310,
        0.3499, 0.3352, 0.3904, 0.3824, 0.2467, 0.4338, 0.3995, 0.4271, 0.2922,
        0.4578, 0.3926, 0.3560, 0.3758, 0.4133, 0.3498, 0.3788, 0.3813, 0.3621,
        0.3929, 0.3351, 0.5983, 0.3515, 0.6105, 0.3572, 0.3009, 0.4728, 0.3892,
        0.3242, 0.4289, 0.3960, 0.3981, 

In [17]:
len(cos_scores)

351

In [18]:
torch.sort(cos_scores,  descending=True)

torch.return_types.sort(
values=tensor([0.6641, 0.6397, 0.6105, 0.6041, 0.5995, 0.5983, 0.5926, 0.5829, 0.5794,
        0.5565, 0.5553, 0.5550, 0.5524, 0.5441, 0.5374, 0.5340, 0.5322, 0.5274,
        0.5272, 0.5165, 0.5161, 0.5157, 0.5154, 0.5128, 0.5100, 0.5053, 0.5010,
        0.5006, 0.4982, 0.4955, 0.4927, 0.4906, 0.4901, 0.4898, 0.4877, 0.4872,
        0.4867, 0.4862, 0.4859, 0.4845, 0.4843, 0.4836, 0.4823, 0.4823, 0.4771,
        0.4744, 0.4728, 0.4684, 0.4675, 0.4668, 0.4657, 0.4638, 0.4623, 0.4617,
        0.4603, 0.4596, 0.4586, 0.4578, 0.4570, 0.4557, 0.4515, 0.4462, 0.4453,
        0.4393, 0.4389, 0.4384, 0.4382, 0.4376, 0.4366, 0.4363, 0.4356, 0.4338,
        0.4316, 0.4310, 0.4299, 0.4295, 0.4289, 0.4271, 0.4257, 0.4242, 0.4225,
        0.4205, 0.4193, 0.4185, 0.4141, 0.4134, 0.4133, 0.4132, 0.4128, 0.4123,
        0.4103, 0.4096, 0.4095, 0.4095, 0.4093, 0.4092, 0.4091, 0.4091, 0.4059,
        0.4048, 0.4048, 0.4039, 0.4037, 0.4032, 0.4029, 0.4017, 0.4006, 0.3997,
        

In [19]:
sbert_result_index

[8639,
 43893,
 115408,
 7847,
 65988,
 13178,
 125472,
 93664,
 132567,
 14948,
 23549,
 125465,
 83375,
 128803,
 133663,
 43892,
 23461,
 5767,
 14959,
 43895]

In [20]:
import pandas as pd
tatal_laws_data = pd.read_csv('total.csv')

In [21]:
result_law_dic = {} #{"우체국보험특별회계법 시행규칙":[123513,12345,234,123]}
for i in sbert_result_index:
    index_num = int(i)
    #print(laws_data_list_cut[index_num])
    data_detail = tatal_laws_data.iloc[[index_num],:]
    # print(tatal_laws_data.iloc[index_num]["법령명"])
    
    law_name = tatal_laws_data.iloc[index_num]["법령명"]
    print('법령명 : ',tatal_laws_data.iloc[index_num]["법령명"])
    if law_name in result_law_dic:
        value_list = result_law_dic[law_name] 
        value_list.append(index_num)
        
    else:
        result_law_dic[law_name] = [index_num]
            

법령명 :  고등교육법
법령명 :  대학 등록금에 관한 규칙
법령명 :  지방공무원 임용후보자 장학규정
법령명 :  경찰대학 설치법
법령명 :  사학기관 재무ㆍ회계 규칙에 대한 특례규칙
법령명 :  공무원채용후보자장학규정
법령명 :  취업 후 학자금 상환 특별법
법령명 :  우체국보험특별회계법 시행령
법령명 :  한국장학재단 설립 등에 관한 법률
법령명 :  공중보건장학을 위한 특례법 시행령
법령명 :  국립대학의 회계 설치 및 재정 운영에 관한 법률 시행규칙
법령명 :  취업 후 학자금 상환 특별법
법령명 :  숙련기술장려법 시행령
법령명 :  폐기물관리법
법령명 :  한옥 등 건축자산의 진흥에 관한 법률
법령명 :  대학 등록금에 관한 규칙
법령명 :  국립대학법인 인천대학교 설립ㆍ운영에 관한 법률 시행규칙
법령명 :  건축서비스산업 진흥법
법령명 :  공중보건장학을 위한 특례법 시행규칙
법령명 :  대학 등록금에 관한 규칙


In [39]:
result_law_dic

{'고등교육법': [8639],
 '대학 등록금에 관한 규칙': [43893, 43892, 43895],
 '지방공무원 임용후보자 장학규정': [115408],
 '경찰대학 설치법': [7847],
 '사학기관 재무ㆍ회계 규칙에 대한 특례규칙': [65988],
 '공무원채용후보자장학규정': [13178],
 '취업 후 학자금 상환 특별법': [125472, 125465],
 '우체국보험특별회계법 시행령': [93664],
 '한국장학재단 설립 등에 관한 법률': [132567],
 '공중보건장학을 위한 특례법 시행령': [14948],
 '국립대학의 회계 설치 및 재정 운영에 관한 법률 시행규칙': [23549],
 '숙련기술장려법 시행령': [83375],
 '폐기물관리법': [128803],
 '한옥 등 건축자산의 진흥에 관한 법률': [133663],
 '국립대학법인 인천대학교 설립ㆍ운영에 관한 법률 시행규칙': [23461],
 '건축서비스산업 진흥법': [5767],
 '공중보건장학을 위한 특례법 시행규칙': [14959]}

In [44]:
result_law_dic.keys

<built-in method keys of dict object at 0x000001CC37008540>


In [45]:
choice_law_name_value = result_law_dic['대학 등록금에 관한 규칙']

In [47]:
for i in choice_law_name_value:
   
    #print(laws_data_list_cut[index_num])
    data_detail = tatal_laws_data.iloc[[i],:]         
    print('법령명 : ',tatal_laws_data.iloc[i]["법령명"])
    print('법령MST : ',tatal_laws_data.iloc[i]["법령MST"])
    print('법령ID : ', tatal_laws_data.iloc[i]["법령ID"])
    print('시행일자 : ', tatal_laws_data.iloc[i]["시행일자"])
    print('공포번호 : ', tatal_laws_data.iloc[i]["공포번호"])
    print('법령구분명 : ', tatal_laws_data.iloc[i]["법령구분명"])
    print('조문번호 : ', tatal_laws_data.iloc[i]["조문번호"])
    print('조문 : ', tatal_laws_data.iloc[i]["조문내용"])
    print("")

법령명 :  대학 등록금에 관한 규칙
법령MST :  240223
법령ID :  10408
시행일자 :  2022.2.7
공포번호 :  제257호
법령구분명 :  교육부령
조문번호 :  4
조문 :  제4조(징수방법)
 ①대학ㆍ교육대학ㆍ전문대학, 방송대학ㆍ통신대학ㆍ방송통신대학 및 기술대학의 등록금(입학금은 제외한다. 이하 이 조에서 같다)은 학칙이 정하는 바에 따라 학점별ㆍ학기별 또는 월별로 징수할 수 있다. <개정 2008.9.29, 2010.12.2>
				  ②산업대학 및 사이버대학의 등록금은 신청 학점별로 징수하되, 학교의 장은 필요한 경우에 따로 정하여 징수할 수 있다. <개정 2008.9.29, 2010.12.2>
				  ③각종학교의 등록금은 학칙이 정하는 바에 따라 제1항 및 제2항에 준하여 이를 징수한다. <개정 2010.12.2>
				  ④입학금은 학생의 입학시에 전액을 징수한다.
				  ⑤학교는 학기가 개시되기 전(학기가 시작된 이후 학교의 장이 따로 정하는 미등록 휴학기간이 있는 경우에는 그 기간을 말한다)에 휴학하고자 하는 자에 대하여는 등록금을 징수하지 않는다. <개정 2010.12.2>
				  ⑥「고등교육법 시행령」 제53조에 따른 시간제 등록생의 등록금은 신청학점별로 징수하되, 필요한 경우에는 학기별 또는 월별로 징수할 수 있다. <개정 2010.12.2>
				  ⑦제1항부터 제3항까지의 규정에도 불구하고 수업연한이 지났으나 졸업에 필요한 소요학점을 취득하지 못하여 수강등록을 하는 학생에 대한 등록금은 다음 각 호의 기준을 초과하지 않는 범위에서 학교의 장이 정하는 바에 따라 징수한다. <개정 2010.12.2, 2021.2.2>
				1. 학사학위 이하의 과정 가. 1학점부터 3학점까지는 해당 학기 등록금의 6분의 1 해당액						
					 나. 4학점부터 6학점까지는 해당 학기 등록금의 3분의 1 해당액						
					 다. 7학점부터 9학점까지는 해당 학기 등록금의 2분의 1 해당액						
	

In [49]:
choice_law_name_value = result_law_dic['경찰대학 설치법']

In [33]:
for i in choice_law_name_value:
    #print(laws_data_list_cut[index_num])
    data_detail = tatal_laws_data.iloc[[i],:]   
    
    print('법령명 : ',tatal_laws_data.iloc[i]["법령명"])
    print('법령MST : ',tatal_laws_data.iloc[i]["법령MST"])
    print('법령ID : ', tatal_laws_data.iloc[i]["법령ID"])
    print('시행일자 : ', tatal_laws_data.iloc[i]["시행일자"])
    print('공포번호 : ', tatal_laws_data.iloc[i]["공포번호"])
    print('법령구분명 : ', tatal_laws_data.iloc[i]["법령구분명"])
    print('조문번호 : ', tatal_laws_data.iloc[i]["조문번호"])
    print('조문 : ', tatal_laws_data.iloc[i]["조문내용"])
    print("")

법령명 :  경찰대학 설치법
법령MST :  224453
법령ID :  990
시행일자 :  2021.1.1
공포번호 :  제17689호
법령구분명 :  법률
조문번호 :  9
조문 :  제9조(학비 및 수당 지급 등)
 ① 경찰대학 학생(치안대학원 학생은 제외한다. 이하 이 조에서 같다)의 학비는 전액 국고에서 부담한다. <개정 2016.5.29>
				  ② 경찰대학 학생에게는 수당과 의복, 그 밖에 교육에 필요한 물품을 지급하고 급식을 한다.
				  ③ 학장은 치안대학원에 입학한 학생에게 수업료와 그 밖의 납부금(이하 "등록금"이라 한다)을 받을 수 있다. <신설 2016.5.29>
				  ④ 제3항에 따른 등록금의 징수에 필요한 사항은 대통령령으로 정한다. <신설 2016.5.29>
				

